In [53]:
import numpy as np
import h5py
import os,sys

In [94]:
def readh5(filename, datasetname='main', rr=[1,1,1]):
    fid=h5py.File(filename,'r')
    if isinstance(datasetname, (list,)):
        out = [None] *len(datasetname)
        for i,dd in enumerate(datasetname):
            sz = len(fid[dd].shape)
            if sz==2:
                out[i] = np.array(fid[dd][::rr[0],::rr[1]])
            elif sz==3:
                out[i] = np.array(fid[dd][::rr[0],::rr[1],::rr[2]])
    else:
        sz = len(fid[datasetname].shape)
        if sz==2:
            out = np.array(fid[datasetname][::rr[0],::rr[1]])
        elif sz==3:
            out = np.array(fid[datasetname][::rr[0],::rr[1],::rr[2]])
    return out

def writeh5(filename, dtarray, datasetname='main'): 
    fid=h5py.File(filename,'w')                                                                      
    if isinstance(datasetname, (list,)):                                                             
        for i,dd in enumerate(datasetname):                                                          
            ds = fid.create_dataset(dd, dtarray[i].shape, compression="gzip", dtype=dtarray[i].dtype)
            ds[:] = dtarray[i]                                                                       
    else:                                                                                            
        ds = fid.create_dataset(datasetname, dtarray.shape, compression="gzip", dtype=dtarray.dtype) 
        ds[:] = dtarray                                                                              
    fid.close()
    
def squeezeto255(dtarray):
    """
    To process Dyer17's dataset.
    Given a volume, scale all the voxels into the range of (0, 255).
    """
    volume_min = np.min(dtarray)
    # First make all voxel positive, then make the range into (0, 255) 
    volume = -1.0 * volume_min + dtarray
    volume_max = np.max(volume)
    volume = (255.0 / volume_max) * volume
    return volume
    

In [95]:
V1volume = readh5('V1_img.h5')

In [96]:
V1volume_squeezed = squeezeto255(V1volume) #min=-200, max=105

In [101]:
writeh5('V1_img_255.h5', V1volume_squeezed)

In [102]:
V1volume = readh5('V1_img_255.h5')